In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.transforms import autoaugment
from PIL import Image
import time
import pickle
import wandb

import wandb
wandb.login()
    

True

In [ ]:
#  1. Data Loading and Preprocessing with configurable augmentation
cifar10_dir = 'deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/'
def load_cifar_batch(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

from PIL import Image
class CustomTestDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, idx  # Return index as a placeholder for label
class CustomCIFAR10Dataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images            # e.g. a NumPy array of shape (N, 32, 32, 3)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]         # This is a NumPy array
        label = self.labels[idx]

        # Convert to PIL if your transforms expect PIL images:
        # (If your transforms are purely tensor-based and include
        #  transforms.ToTensor() as the first step, you can skip this
        #  and let ToTensor() handle the NumPy array directly.)
        img = Image.fromarray(img)

        if self.transform:
            img = self.transform(img)

        return img, label

    
def get_data_loaders(batch_size=128, num_workers=0, config=None):
    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
    
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])
    
    # Use wandb config to control the parameters
    train_transform_list = [transforms.ToTensor(), normalize]
    if config.random_crop:
        train_transform_list.insert(0, transforms.RandomCrop(32, padding=4))
    if config.horizontal_flip:
        train_transform_list.insert(0, transforms.RandomHorizontalFlip())
    train_transform_list.insert(0, transforms.ColorJitter(
        brightness=config.color_jitter_brightness,
        contrast=0.2,
        saturation=0.2,
        hue=0.1
    ))
    train_transform_list.insert(0, autoaugment.AutoAugment(policy=autoaugment.AutoAugmentPolicy.CIFAR10))
    train_transform_list.append(transforms.RandomErasing(
        p=config.random_erasing_prob,
        scale=(0.02, 0.33),
        ratio=(0.3, 3.3),
        value=0
    ))
    
    train_transform = transforms.Compose(train_transform_list)

    # Specify the directory containing CIFAR-10 batches
    cifar10_dir = 'deep-learning-spring-2025-project-1/cifar-10-python/cifar-10-batches-py/'

    # Load metadata (labels)
    meta_data_dict = load_cifar_batch(os.path.join(cifar10_dir, 'batches.meta'))
    label_names = [label.decode('utf-8') for label in meta_data_dict[b'label_names']]

    # Load training data
    train_data = []
    train_labels = []
    for i in range(1, 6):
        batch = load_cifar_batch(os.path.join(cifar10_dir, f'data_batch_{i}'))
        train_data.append(batch[b'data'])
        train_labels += batch[b'labels']

    train_data = np.vstack(train_data).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Convert to HWC format
    train_labels = np.array(train_labels)

    # Load test data
    batch_test_dict = load_cifar_batch(os.path.join(cifar10_dir, 'test_batch'))
    val_images = batch_test_dict[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    val_labels = np.array(batch_test_dict[b'labels'])
    
    #get the datasets
    train_dataset = CustomCIFAR10Dataset(train_data, train_labels, transform=train_transform)
    test_dataset = CustomCIFAR10Dataset(val_images, val_labels, transform=test_transform)

    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size,
        shuffle=True, 
        num_workers=num_workers
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size,
        shuffle=False, 
        num_workers=num_workers
    )
    
    return train_loader, test_loader

In [ ]:
# 2. Model Definition (unchanged)
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")
    return total_params, trainable_params

# def get_resnet18_model(num_classes=10, pretrained=False):
#     model = models.resnet18(weights=weights)
#     model.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
#     model.bn1 = nn.BatchNorm2d(32)
#     model.maxpool = nn.Identity()
    
#     model.layer1[0].conv1 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer1[0].conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer1[0].bn1 = nn.BatchNorm2d(32)
#     model.layer1[0].bn2 = nn.BatchNorm2d(32)
#     model.layer1[1].conv1 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer1[1].conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer1[1].bn1 = nn.BatchNorm2d(32)
#     model.layer1[1].bn2 = nn.BatchNorm2d(32)
    
#     model.layer2[0].conv1 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, bias=False)
#     model.layer2[0].conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer2[0].bn1 = nn.BatchNorm2d(64)
#     model.layer2[0].bn2 = nn.BatchNorm2d(64)
#     model.layer2[0].downsample[0] = nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False)
#     model.layer2[0].downsample[1] = nn.BatchNorm2d(64)
#     model.layer2[1].conv1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer2[1].conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer2[1].bn1 = nn.BatchNorm2d(64)
#     model.layer2[1].bn2 = nn.BatchNorm2d(64)
    
#     model.layer3[0].conv1 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, bias=False)
#     model.layer3[0].conv2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer3[0].bn1 = nn.BatchNorm2d(128)
#     model.layer3[0].bn2 = nn.BatchNorm2d(128)
#     model.layer3[0].downsample[0] = nn.Conv2d(64, 128, kernel_size=1, stride=2, bias=False)
#     model.layer3[0].downsample[1] = nn.BatchNorm2d(128)
#     model.layer3[1].conv1 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer3[1].conv2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer3[1].bn1 = nn.BatchNorm2d(128)
#     model.layer3[1].bn2 = nn.BatchNorm2d(128)
    
#     model.layer4[0].conv1 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1, bias=False)
#     model.layer4[0].conv2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer4[0].bn1 = nn.BatchNorm2d(256)
#     model.layer4[0].bn2 = nn.BatchNorm2d(256)
#     model.layer4[0].downsample[0] = nn.Conv2d(128, 256, kernel_size=1, stride=2, bias=False)
#     model.layer4[0].downsample[1] = nn.BatchNorm2d(256)
#     model.layer4[1].conv1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer4[1].conv2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False)
#     model.layer4[1].bn1 = nn.BatchNorm2d(256)
#     model.layer4[1].bn2 = nn.BatchNorm2d(256)

    
    # model.fc = nn.Linear(256, num_classes)
    # return model
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.skip = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.skip = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.skip(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += identity
        out = self.relu(out)
        return out

#modified resnet34 structure
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.init_conv = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1, bias=False)
        self.init_bn = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(32, 32, 3, stride=1)
        self.layer2 = self._make_layer(32, 64, 4, stride=2)
        self.layer3 = self._make_layer(64, 128, 5, stride=2)
        self.layer4 = self._make_layer(128, 256, 2, stride=2)

        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = [ResidualBlock(in_channels, out_channels, stride)]
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.init_conv(x)
        out = self.init_bn(out)
        out = self.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


In [18]:
# 3. Mixup Function (unchanged)
def mixup_data(batch, targets, alpha=1.0, device='cpu'):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = batch.size()[0]
    index = torch.randperm(batch_size).to(device)

    mixed_x = lam * batch + (1 - lam) * batch[index]
    y_a, y_b = targets, targets[index]
    
    return mixed_x, y_a, y_b, lam

In [19]:
# 4. Training and Evaluation Functions with wandb logging
def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch, use_mixup=False, mixup_alpha=1.0):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    start_time = time.time()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        if use_mixup:
            inputs, targets_a, targets_b, lam = mixup_data(inputs, targets, alpha=mixup_alpha, device=device)
            outputs = model(inputs)
            loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
        else:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if not use_mixup:
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    epoch_time = time.time() - start_time
    epoch_loss = running_loss / len(train_loader)
    
    if not use_mixup:
        epoch_acc = 100.0 * correct / total
        wandb.log({"train_loss": epoch_loss, "train_accuracy": epoch_acc, "epoch": epoch})
        print(f'Epoch: {epoch}, Train Loss: {epoch_loss:.3f}, Train Acc: {epoch_acc:.2f}%')
        return epoch_loss, epoch_acc
    else:
        wandb.log({"train_loss": epoch_loss, "epoch": epoch})
        print(f'Epoch: {epoch}, Train Loss: {epoch_loss:.3f}')
        return epoch_loss, None
    
def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    test_loss = test_loss / len(test_loader)
    test_acc = 100.0 * correct / total
    wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})
    print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')
    
    return test_loss, test_acc

In [ ]:
# # 5. Main Training Function with MPS Support
# def train(config=None):
#     with wandb.init(config=config):
#         config = wandb.config
        
#         # 设备检测：优先 MPS，然后 CUDA，最后 CPU
#         if torch.backends.mps.is_available():
#             device = torch.device("mps")
#             print(f"Using Apple MPS for acceleration")
#         elif torch.cuda.is_available():
#             device = torch.device("cuda")
#             print(f"Using CUDA for acceleration")
#         else:
#             device = torch.device("cpu")
#             print(f"Using CPU (no GPU/MPS available)")
        
#         # 获取数据加载器
#         train_loader, test_loader = get_data_loaders(batch_size=128, config=config)
        
#         # 初始化模型并移动到指定设备
#         model = get_resnet18_model(num_classes=10).to(device)
#         count_parameters(model)
        
#         # 定义损失函数和优化器
#         criterion = nn.CrossEntropyLoss()
#         optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        
#         # 获取训练的总轮数
#         num_epochs = config.num_epochs if hasattr(config, 'num_epochs') else 20
        
#         # 调整学习率调度器的最大周期为训练轮数
#         scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
        
#         # 训练循环
#         for epoch in range(num_epochs):
#             train_loss, _ = train_one_epoch(
#                 model, train_loader, criterion, optimizer, device, epoch+1,
#                 use_mixup=True, mixup_alpha=config.mixup_alpha
#             )
#             test_loss, test_acc = evaluate(model, test_loader, criterion, device)
#             scheduler.step()

# 5. Main Training Function with MPS Support

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        # Get the unique run ID to use in filenames
        run_id = wandb.run.id
        
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print("Using CUDA for acceleration")
        else:
            device = torch.device("cpu")
            print("Using CPU (no GPU available)")
        
        # getting dataloaders
        train_loader, test_loader = get_data_loaders(batch_size=128, config=config)
        
        # Initialize the model and move it to the selected device
        model = ResNet18(num_classes=10).to(device)
        count_parameters(model)
        
        # cruterion and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        
        # number of epoch we train
        num_epochs = config.num_epochs if hasattr(config, 'num_epochs') else 20
        
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
        
        # tracking the best model
        best_acc = 0.0
        
        # Create model filenames with run ID
        best_model_path = f'best_model_{run_id}.pth'
        final_model_path = f'final_model_{run_id}.pth'
        
        # loop for train
        for epoch in range(num_epochs):
            train_loss, _ = train_one_epoch(
                model, train_loader, criterion, optimizer, device, epoch+1,
                use_mixup=True, mixup_alpha=config.mixup_alpha
            )
            test_loss, test_acc = evaluate(model, test_loader, criterion, device)
            scheduler.step()
            
            # make sure the best model
            if test_acc > best_acc:
                best_acc = test_acc
                # save the model to local
                torch.save(model.state_dict(), best_model_path)
                # save to wandb
                wandb.save(best_model_path)
                
                # Also log the best accuracy to wandb config for easy retrieval
                wandb.run.summary['best_accuracy'] = best_acc
        
        # save the final model
        torch.save(model.state_dict(), final_model_path)
        wandb.save(final_model_path)
        
        # Log final test accuracy
        wandb.run.summary['final_accuracy'] = test_acc
        
        # Also save a record of which model is best for this run
        with open(f'model_info_{run_id}.txt', 'w') as f:
            f.write(f"Best model: {best_model_path}, Accuracy: {best_acc:.2f}%\n")
            f.write(f"Final model: {final_model_path}, Accuracy: {test_acc:.2f}%\n")
            f.write("\nRun Configuration:\n")
            for key, value in config._items.items():
                f.write(f"{key}: {value}\n")
        wandb.save(f'model_info_{run_id}.txt')

In [ ]:
# 6. Sweep Configuration and Execution
if __name__ == '__main__':
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    
    wandb.login()
    sweep_config = {
        "method": "grid",
        "metric": {"name": "test_accuracy", "goal": "maximize"},
        "parameters": {
            "mixup_alpha": {"values": [0.2]},
            "random_crop": {"values": [True]},
            "horizontal_flip": {"values": [True]}, #Horizontal Flip really help！
            "color_jitter_brightness": {"values": [0.2]},
            "random_erasing_prob": {"values": [0.2]},
            "num_epochs": {"values": [150]}  # Add this line to set num_epochs
        }
    }
    
    sweep_id = wandb.sweep(sweep_config, project="cifar10-augmentation-sweep")
    # wandb.agent(sweep_id, train, count=10)  # count=20 mean run 20 experiments
    wandb.agent(sweep_id, train)  # no need of count. automatically run all the combinations

Create sweep with ID: r9ye1bdz
Sweep URL: https://wandb.ai/kp2653-new-york-university/cifar10-augmentation-sweep/sweeps/r9ye1bdz


wandb: Agent Starting Run: xwg1cm8l with config:
wandb: 	color_jitter_brightness: 0.2
wandb: 	horizontal_flip: True
wandb: 	mixup_alpha: 0.2
wandb: 	num_epochs: 150
wandb: 	random_crop: True
wandb: 	random_erasing_prob: 0.2
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Using CUDA for acceleration
Total parameters: 3,850,410
Trainable parameters: 3,850,410
Epoch: 1, Train Loss: 2.186
Test Loss: 2.048, Test Acc: 33.32%
Epoch: 2, Train Loss: 1.948
Test Loss: 1.661, Test Acc: 36.98%
Epoch: 3, Train Loss: 1.795
Test Loss: 1.424, Test Acc: 49.53%
Epoch: 4, Train Loss: 1.660
Test Loss: 1.251, Test Acc: 55.60%
Epoch: 5, Train Loss: 1.533
Test Loss: 1.152, Test Acc: 60.27%
Epoch: 6, Train Loss: 1.443
Test Loss: 0.978, Test Acc: 65.70%
Epoch: 7, Train Loss: 1.372
Test Loss: 1.032, Test Acc: 65.86%
Epoch: 8, Train Loss: 1.259
Test Loss: 1.015, Test Acc: 65.94%
Epoch: 9, Train Loss: 1.247
Test Loss: 0.777, Test Acc: 74.04%
Epoch: 10, Train Loss: 1.231
Test Loss: 0.889, Test Acc: 70.18%
Epoch: 11, Train Loss: 1.270
Test Loss: 0.918, Test Acc: 69.22%
Epoch: 12, Train Loss: 1.203
Test Loss: 0.818, Test Acc: 73.79%
Epoch: 13, Train Loss: 1.164
Test Loss: 0.682, Test Acc: 76.56%
Epoch: 14, Train Loss: 1.193
Test Loss: 0.763, Test Acc: 75.04%
Epoch: 15, Train Loss: 1.

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
test_accuracy,▁▂▃▅▄▅▃▅▅▆▅▆▅▆▅▆▅▆▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇███████
test_loss,█▇▅▄▃▃▃▃▃▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁
train_loss,█▇▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▃▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁
best_accuracy,95.04
epoch,150
final_accuracy,95.03
test_accuracy,95.03
test_loss,0.17746
train_loss,0.5703


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [22]:
# # Generate submission file
import torch
import pandas as pd
import numpy as np
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ResNet18(num_classes=10)
model.load_state_dict(torch.load('best_model_xwg1cm8l.pth', map_location=device))
model.to(device)
model.eval()
cifar_test_path = 'deep-learning-spring-2025-project-1/cifar_test_nolabel.pkl'
test_batch = load_cifar_batch(cifar_test_path)
test_images = test_batch[b'data'].astype(np.float32) / 255.0

# Convert test dataset to Tensor
test_dataset = [(test_transform(img),) for img in test_images]
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)
predictions = []
with torch.no_grad():
    for batch in test_loader:
        images = batch[0].to(device)  # Get images tensor from tuple and move to device
        outputs = model(images) 
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())

# Generate submission file
submission = pd.DataFrame({'ID': np.arange(len(predictions)), 'Labels': predictions})
submission.to_csv('submission.csv', index=False)
print("Submission file saved.")

Submission file saved.


In [ ]:
# 7. Prediction on Test Dataset
import pickle
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import wandb
import os
import glob

class CustomTestDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, idx  # Return index as a placeholder for label

def predict_test_dataset():
    # Load the test dataset
    with open('deep-learning-spring-2025-project-1/cifar_test_nolabel.pkl', 'rb') as f:
        test_data = pickle.load(f)
    
    # Define the same transformations used for testing
    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2470, 0.2435, 0.2616]
        )
    ])
    
    # Create test dataset and dataloader
    test_dataset = CustomTestDataset(test_data, transform=test_transform)
    test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)
    
    # Set device
    if torch.cuda.is_available():
            device = torch.device("cuda")
            print("Using CUDA for acceleration")
    else:
        device = torch.device("cpu")
        print("Using CPU (no GPU available)")
    
    # Initialize model using the same architecture as in training
    model = ResNet18(num_classes=10).to(device)
    
    # Try to find the best model from all runs
    try:
        api = wandb.Api()
        runs = api.runs("cifar10-augmentation-sweep")
        best_run = None
        best_accuracy = 0
        
        for run in runs:
            if run.state == "finished" and run.summary.get("best_accuracy", 0) > best_accuracy:
                best_accuracy = run.summary.get("best_accuracy", 0)
                best_run = run
        
        if best_run:
            run_id = best_run.id
            best_model_path = f'best_model_{run_id}.pth'
            print(f"Using best model from run {best_run.name} with accuracy {best_accuracy:.2f}%")
            
            try:
                # Try to download the model file
                model_file = best_run.file(best_model_path).download(replace=True)
                model.load_state_dict(torch.load(model_file.name, map_location=device))
                print(f"Successfully loaded model from wandb")
            except Exception as e:
                print(f"Error downloading model from wandb: {e}")
                
                # Try to find the model locally
                if os.path.exists(best_model_path):
                    model.load_state_dict(torch.load(best_model_path, map_location=device))
                    print(f"Loaded model from local file: {best_model_path}")
                else:
                    print(f"Could not find model file: {best_model_path}")
                    
                    # Try to find any best model file locally
                    best_models = glob.glob('best_model_*.pth')
                    if best_models:
                        latest_model = max(best_models, key=os.path.getctime)
                        model.load_state_dict(torch.load(latest_model, map_location=device))
                        print(f"Loaded most recent local model: {latest_model}")
                    else:
                        print("No model files found locally")
    except Exception as e:
        print(f"Error accessing wandb: {e}")
        
        # Try to find any best model file locally
        best_models = glob.glob('best_model_*.pth')
        if best_models:
            latest_model = max(best_models, key=os.path.getctime)
            model.load_state_dict(torch.load(latest_model, map_location=device))
            print(f"Loaded most recent local model: {latest_model}")
        else:
            print("WARNING: Could not load any model. Using untrained model for predictions.")
    
    model.eval()
    


# Run the prediction function
predict_test_dataset()

Using CUDA for acceleration
Using best model from run major-sweep-1 with accuracy 95.04%
Successfully loaded model from wandb


KeyError: 0

In [ ]:
# import pickle
# import pandas as pd
# import torch
# from torchvision import transforms
# from torch.utils.data import DataLoader, Dataset
# import os
# import glob
# from PIL import Image

# def predict_test_dataset(model_path=None):
#     print("Starting prediction function...")
    
#     # Load the test dataset
#     print("Loading test data...")
#     with open('/Users/JL/Desktop/DL_Proj/cifar_test_nolabel.pkl', 'rb') as f:
#         test_data = pickle.load(f)
#     print("Test data loaded successfully!")
    
#     # Define the same transformations used for testing
#     test_transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(
#             mean=[0.4914, 0.4822, 0.4465],
#             std=[0.2470, 0.2435, 0.2616]
#         )
#     ])
    
#     # Create a proper dataset based on the actual structure
#     class CIFARTestDataset(Dataset):
#         def __init__(self, data_dict, transform=None):
#             self.images = data_dict[b'data']  # Already in shape (10000, 32, 32, 3)
#             self.ids = data_dict[b'ids']
#             self.transform = transform
            
#         def __len__(self):
#             return len(self.images)
        
#         def __getitem__(self, idx):
#             image = self.images[idx]
#             image_id = int(self.ids[idx])  # Convert to int for DataFrame
            
#             # Convert to PIL Image for transforms
#             image = Image.fromarray(image)
            
#             if self.transform:
#                 image = self.transform(image)
                
#             return image, image_id
    
#     # Create test dataset and dataloader
#     print("Creating dataset and dataloader...")
#     test_dataset = CIFARTestDataset(test_data, transform=test_transform)
#     test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)
#     print(f"Dataset created with {len(test_dataset)} samples")
    
#     # Set device
#     print("Setting up device...")
#     if torch.backends.mps.is_available():
#         device = torch.device("mps")
#         print(f"Using Apple MPS for acceleration")
#     elif torch.cuda.is_available():
#         device = torch.device("cuda")
#         print(f"Using CUDA for acceleration")
#     else:
#         device = torch.device("cpu")
#         print(f"Using CPU (no GPU/MPS available)")
    
#     # Initialize model using the same architecture as in training
#     print("Initializing model...")
#     model = get_resnet18_model(num_classes=10).to(device)
#     print("Model initialized")
    
#     # Load the specified model if provided
#     print("Loading model weights...")
#     if model_path and os.path.exists(model_path):
#         model.load_state_dict(torch.load(model_path, map_location=device))
#         print(f"Successfully loaded model from: {model_path}")
#     else:
#         # If no specific model is provided, try to find the best model
#         if model_path:
#             print(f"Specified model {model_path} not found.")
        
#         # Try to find any best model file locally
#         best_models = glob.glob('best_model_*.pth')
#         if best_models:
#             latest_model = max(best_models, key=os.path.getctime)
#             model.load_state_dict(torch.load(latest_model, map_location=device))
#             print(f"Loaded most recent local model: {latest_model}")
#         else:
#             print("WARNING: Could not load any model. Using untrained model for predictions.")
    
#     # Set model to evaluation mode
#     model.eval()
#     print("Model set to evaluation mode")
    
#     # Make predictions
#     print("Making predictions...")
#     predictions = []
#     with torch.no_grad():
#         for batch_idx, (images, ids) in enumerate(test_loader):
#             print(f"Processing batch {batch_idx+1}/{len(test_loader)}")
#             images = images.to(device)
#             outputs = model(images)
#             _, preds = torch.max(outputs, 1)
            
#             for id_val, pred in zip(ids.numpy(), preds.cpu().numpy()):
#                 predictions.append((id_val, pred))
    
#     # Create submission DataFrame
#     print("Creating submission file...")
#     submission_df = pd.DataFrame(predictions, columns=['ID', 'Labels'])
    
#     # Save to CSV
#     submission_df.to_csv('submission.csv', index=False)
#     print(f"Predictions saved to submission.csv")
#     print("Prediction function completed successfully!")

# # Call the function with your model path
# print("About to call prediction function...")
# predict_test_dataset("/Users/JL/Desktop/DL_Proj/final_model_ku8uu15z.pth")
# print("Prediction function call completed.")

In [ ]:
# import pickle
# import pandas as pd
# import torch
# from torchvision import transforms
# from torch.utils.data import DataLoader, Dataset
# import os
# import glob
# from PIL import Image

# def debug_test_dataset():
#     print("Starting debug function...")
    
#     try:
#         # Load the test dataset
#         print("Attempting to load test data...")
#         with open('/Users/JL/Desktop/DL_Proj/cifar_test_nolabel.pkl', 'rb') as f:
#             test_data = pickle.load(f)
#         print("Test data loaded successfully!")
        
#         # Print test data structure for debugging
#         print(f"Test data type: {type(test_data)}")
        
#         if isinstance(test_data, dict):
#             print(f"Test data keys: {list(test_data.keys())}")
#         elif isinstance(test_data, list):
#             print(f"Test data is a list with {len(test_data)} items")
#         else:
#             print(f"Test data is of type {type(test_data)}")
            
#         print("Debug function completed successfully!")
#         return test_data
#     except Exception as e:
#         print(f"Error in debug function: {e}")
#         import traceback
#         traceback.print_exc()
#         return None

# # Call the debug function
# test_data = debug_test_dataset()

# # If we got data, try to examine it further
# if test_data is not None:
#     print("\nExamining test data further...")
#     if isinstance(test_data, dict):
#         for key in test_data.keys():
#             print(f"Key: {key}, Type: {type(test_data[key])}")
#             if isinstance(test_data[key], (list, tuple, np.ndarray)):
#                 print(f"Length/Shape: {len(test_data[key]) if isinstance(test_data[key], (list, tuple)) else test_data[key].shape}")